# emulator-to-json
notebook for making json of network error for each parameter for use in likelihood estimation later on

In [1]:
#### misc
import pandas as pd
import numpy as np
import os
from pathlib import Path
import pickle
import time
from itertools import product
from scipy import constants
from scipy import stats
import random
import astropy.constants
from numpy.random import default_rng
import json

#### graphical
import matplotlib.pyplot as plt

#### ML
import sklearn
from sklearn.decomposition import PCA
import tensorflow as tf
import keras
from keras import layers

#### custom
from InversePCA import InversePCA
from WMSE import WMSE, WMSE_metric

##### poke gpu
os.environ["CUDA_VISIBLE_DEVICES"]="1"

physical_devices = tf.config.list_physical_devices("GPU") 

gpu0usage = tf.config.experimental.get_memory_info("GPU:0")["current"]

print("Current GPU usage:\n"
     + " - GPU0: " + str(gpu0usage) + "B\n")

def calc_Teff(luminosity, radius):
    return np.array(((luminosity.values*astropy.constants.L_sun) / (4*np.pi*constants.sigma*((radius.values*astropy.constants.R_sun)**2)))**0.25)

def rescale_preds(preds, df, column):
    if 'star_feh' in column:
        return (preds[column+"_std"]*df[column].std())+df[column].mean()
    else:
        return 10**((preds["log_"+column+"_std"]*df["log_"+column].std())+df["log_"+column].mean())

2024-05-03 10:27:26.581394: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-03 10:27:26.581425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-03 10:27:26.582370: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-03 10:27:26.587065: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-03 10:27:27.125216: W tensorflow/compiler/tf2

Current GPU usage:
 - GPU0: 0B



2024-05-03 10:27:27.606009: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1116 MB memory:  -> device: 0, name: NVIDIA RTX A4500, pci bus id: 0000:61:00.0, compute capability: 8.6


In [2]:
df_full = pd.read_hdf('/home/oxs235/datastorage/repos_data/ojscutt/pitchfork/data/bob.h5', key='df')

#### define inputs
inputs = ['log_initial_mass_std', 'log_initial_Zinit_std', 'log_initial_Yinit_std', 'log_initial_MLT_std', 'log_star_age_std']

#### define outputs
classical_outputs = ['log_radius_std', 'log_luminosity_std', 'star_feh_std']
astero_outputs = [f'log_nu_0_{i+1}_std' for i in range(5,40)]

outputs = classical_outputs+astero_outputs

#### train/test split
seed = 42

df_train = df_full.sample(frac=0.95, random_state=seed)
df_test = df_full.drop(df_train.index)

df_train_inputs, df_val_inputs, df_train_outputs, df_val_outputs = sklearn.model_selection.train_test_split(df_train[inputs],df_train[outputs], test_size = 0.005, random_state=seed)

print("Training set: ", len(df_train_inputs))
print("Testing set: ", len(df_test))
print("Validation set: ", len(df_val_inputs))

#### can't have too many describes
df_test.to_hdf("test_data.h5", index=False,key='df')
df_full.describe()

Training set:  2314615
Testing set:  122434
Validation set:  11632


,initial_mass,initial_Zinit,initial_Yinit,initial_MLT,star_age,radius,luminosity,effective_T,star_feh,nu_0_4,...,log_nu_0_32_std,log_nu_0_33_std,log_nu_0_34_std,log_nu_0_35_std,log_nu_0_36_std,log_nu_0_37_std,log_nu_0_38_std,log_nu_0_39_std,log_nu_0_40_std,log_calc_effective_T_std
count,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,...,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06
mean,1.021885e+00,1.440698e-02,2.814693e-01,2.117984e+00,5.000177e+00,1.312728e+00,2.116482e+00,5.870890e+03,-1.127145e-01,5.459277e+02,...,3.629539e-15,2.700240e-15,1.435547e-15,-4.423639e-15,8.003407e-15,-8.417543e-15,1.289578e-15,-3.185318e-15,3.670395e-15,-4.006495e-14
std,1.175610e-01,9.677123e-03,2.805593e-02,2.879175e-01,3.405371e+00,4.813547e-01,1.572419e+00,5.674285e+02,3.499532e-01,2.021373e+02,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,8.000000e-01,3.869061e-03,2.400000e-01,1.700000e+00,2.966411e-02,6.990236e-01,1.321735e-01,4.099787e+03,-1.077903e+00,1.545097e+02,...,-2.599363e+00,-2.594960e+00,-2.591708e+00,-2.589646e+00,-2.588901e+00,-2.589287e+00,-2.590747e+00,-2.593002e+00,-2.595848e+00,-3.657017e+00
25%,9.200000e-01,6.471429e-03,2.600000e-01,1.900000e+00,2.309880e+00,9.719693e-01,9.371542e-01,5.495672e+03,-4.000000e-01,3.986516e+02,...,-5.643108e-01,-5.676179e-01,-5.705508e-01,-5.723483e-01,-5.730070e-01,-5.725672e-01,-5.713737e-01,-5.700038e-01,-5.685747e-01,-6.332417e-01
50%,1.040000e+00,1.077191e-02,2.800000e-01,2.100000e+00,4.275231e+00,1.160016e+00,1.698469e+00,5.863863e+03,-1.000000e-01,5.591040e+02,...,2.417789e-01,2.415099e-01,2.405877e-01,2.389688e-01,2.369176e-01,2.348740e-01,2.333131e-01,2.326247e-01,2.327753e-01,3.593147e-02
75%,1.120000e+00,2.007835e-02,3.000000e-01,2.300000e+00,7.159688e+00,1.498369e+00,2.883064e+00,6.223793e+03,1.837200e-01,6.947397e+02,...,7.569644e-01,7.578097e-01,7.587334e-01,7.595544e-01,7.599729e-01,7.598420e-01,7.591873e-01,7.580180e-01,7.566985e-01,6.506511e-01
max,1.200000e+00,3.897971e-02,3.200000e-01,2.500000e+00,1.399997e+01,2.873431e+00,1.163902e+01,7.998610e+03,5.094573e-01,1.021990e+03,...,1.617024e+00,1.618137e+00,1.620054e+00,1.622624e+00,1.625764e+00,1.629202e+00,1.632519e+00,1.633559e+00,1.635647e+00,3.239603e+00


In [3]:
"""
pca
"""
#### define pca global vars
n_components = 15
seed = 42

#### define and fit pca
pca = PCA(n_components=n_components, random_state=seed)
pca.fit(df_full[astero_outputs])

#### print variance with chosen n_comps
print("Explained variance ratio with n_comps = " + str(n_components) + " is " + str(sum(pca.explained_variance_ratio_)))

Explained variance ratio with n_comps = 15 is 0.9999999417914724


In [4]:
"""
DEFINE WEIGHTS FOR WMSE
"""
log_weights = (1/np.log(10)) * np.array(([0.01, 0.02, 0.0001]+[0.1 for i in range(len(astero_outputs))]) / df_full[["radius", "luminosity", "star_feh"] + [f'nu_0_{i+1}' for i in range(5,40)]].mean())

log_weights = log_weights / df_full[["log_radius", "log_luminosity", "star_feh"] + [f'log_nu_0_{i+1}' for i in range(5,40)]].std()


weights = log_weights.values.tolist()

weights[2] = 0.01

print(weights)

[0.02324560601067967, 0.012220076304794422, 0.01, 0.000287021912847026, 0.0002479564499609835, 0.00021803793653692188, 0.00019473329437678694, 0.00017636813775390715, 0.00016157824162761058, 0.00014938082044491918, 0.0001390395832897473, 0.00013006146357564936, 0.00012220154533326764, 0.00011530706805035573, 0.000109209187475727, 0.00010373957827574545, 9.877493746324607e-05, 9.424043658607474e-05, 9.008674371702482e-05, 8.626668769588676e-05, 8.273438289926203e-05, 7.944721575439512e-05, 7.636537366843286e-05, 7.346196763120404e-05, 7.073020022961854e-05, 6.817783121908433e-05, 6.58107770189888e-05, 6.362486643635147e-05, 6.160808701834725e-05, 5.974558620227938e-05, 5.8022878771033603e-05, 5.642751783475358e-05, 5.4947516208327664e-05, 5.357098419381567e-05, 5.228562829266241e-05, 5.1078328399597434e-05, 4.993536750819551e-05, 4.8843689480388554e-05]


In [5]:
"""
load in angry mob
"""
pca_comps = pca.components_
pca_mean = pca.mean_

custom_objects = {"InversePCA": InversePCA(pca_comps, pca_mean),"WMSE": WMSE(weights)}

pitchfork_name = "nu6-40_elu_nonorm_feh"
pitchfork = tf.keras.models.load_model("pickle jar/" + pitchfork_name + ".h5", custom_objects=custom_objects)

In [6]:
inputs = ['log_initial_mass_std', 'log_initial_Zinit_std', 'log_initial_Yinit_std', 'log_initial_MLT_std', 'log_star_age_std']

pred_outputs = ['log_radius_std', 'log_luminosity_std', 'star_feh_std'] + [f'log_nu_0_{i+1}_std' for i in range(5,40)]

outputs = ['radius', 'luminosity', 'star_feh'] + [f'nu_0_{i+1}' for i in range(5,40)]

preds = pitchfork(np.array(df_test[inputs]))

preds_df = pd.DataFrame(np.concatenate((np.array(preds[0]),np.array(preds[1])), axis=1), columns=pred_outputs)


for output in outputs:
    preds_df[output] = rescale_preds(preds_df, df_full, output)

preds_df['effective_T'] = calc_Teff(preds_df["luminosity"],preds_df["radius"])

pitchfork_errors_df=pd.DataFrame(np.array(df_test[outputs+['effective_T']]) - np.array(preds_df[outputs+['effective_T']]), columns=[outputs+['effective_T']])

2024-05-03 10:27:51.273142: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [10]:
pitchfork_errors_df = pitchfork_errors_df[['effective_T', 'luminosity', 'star_feh'] + [f'nu_0_{i+1}' for i in range(5,40)]]
pitchfork_errors_df

,effective_T,luminosity,star_feh,nu_0_6,nu_0_7,nu_0_8,nu_0_9,nu_0_10,nu_0_11,nu_0_12,...,nu_0_31,nu_0_32,nu_0_33,nu_0_34,nu_0_35,nu_0_36,nu_0_37,nu_0_38,nu_0_39,nu_0_40
0,1.923269,-0.000165,-0.000484,0.144269,-0.462667,-0.250838,-0.798832,-0.156309,-0.339588,-0.868754,...,0.281421,0.419974,-0.096299,-0.695389,-0.367608,0.807108,1.625223,-2.934542,-2.155523,-1.758358
1,0.620388,-0.000271,-0.000464,-0.008636,-0.700756,-0.446926,-0.914589,-0.223743,-0.341258,-0.884319,...,0.097307,0.122588,-0.421236,-0.952874,-0.349897,0.804834,0.307938,-2.745218,-2.392087,-2.874111
2,0.433352,-0.000274,-0.000472,-0.073690,-0.776345,-0.519323,-0.989458,-0.298677,-0.398918,-0.949791,...,-0.145570,-0.150444,-0.704151,-1.223538,-0.582535,0.562711,-0.357406,-2.940105,-2.653332,-3.269268
3,0.360255,-0.000209,-0.000500,-0.152007,-0.849753,-0.565866,-1.035576,-0.329247,-0.337766,-0.882360,...,-0.325218,-0.412468,-0.979045,-1.412269,-0.624331,0.520714,-3.731763,-2.709889,-2.613945,-3.668847
4,0.588040,-0.000143,-0.000466,-0.122140,-0.802406,-0.505233,-0.973897,-0.253959,-0.213854,-0.747722,...,-0.129025,-0.241967,-0.794085,-1.186079,-0.342994,0.804759,-3.320295,-2.284215,-2.256346,-3.470091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122429,-7.356710,-0.009938,0.000629,0.152768,0.634809,0.320857,0.334960,0.572175,0.214805,0.393452,...,1.901456,2.517938,2.459341,2.169123,2.231622,2.383012,2.318748,2.352560,2.466695,2.566058
122430,4.106118,-0.038118,0.000500,-0.470416,0.470893,0.329083,-0.048767,0.094231,0.018933,-0.524172,...,-0.394664,-0.482586,-0.706038,-0.618565,-0.370062,-0.489049,-0.825498,-0.893396,-0.690679,-0.250742
122431,6.919575,-0.009889,-0.000299,-0.300936,0.543902,-0.077994,-0.225545,0.231112,0.021014,-0.147791,...,0.480300,0.202407,0.021159,0.278962,0.412166,0.253865,0.121367,0.095261,0.160782,0.410327
122432,3.364943,0.022790,0.000521,0.260287,-0.238324,0.048139,-0.040404,-0.039845,-0.179641,0.171434,...,0.404192,0.775037,0.855292,0.529170,0.375255,0.460825,0.689848,0.719678,0.601190,0.470765


In [11]:
pitchfork_errors_df.to_json('emulator_errors.json')

In [46]:
emulator_dict = {}

for column in outputs + ["effective_T"]:
    column_stats =  {'std': np.std(pitchfork_errors_df[column]).values[0], 'mad':stats.median_abs_deviation(pitchfork_errors_df[column])[0]}
    emulator_dict[column]=column_stats

In [47]:
emulator_dict

{'radius': {'std': 0.0031592064924546386, 'mad': 0.0007193065020046352},
 'luminosity': {'std': 0.013091453133521488, 'mad': 0.001662317802538782},
 'star_feh': {'std': 0.0010103521414256134, 'mad': 0.0004036022869300937},
 'nu_0_6': {'std': 0.30991524050630137, 'mad': 0.18503403141781405},
 'nu_0_7': {'std': 0.3576611626309852, 'mad': 0.22224471350972408},
 'nu_0_8': {'std': 0.3701364267757371, 'mad': 0.2275139276885909},
 'nu_0_9': {'std': 0.3276912175745757, 'mad': 0.1971704275900379},
 'nu_0_10': {'std': 0.3607234429548008, 'mad': 0.22212924163957837},
 'nu_0_11': {'std': 0.3354543162205072, 'mad': 0.20247161307119654},
 'nu_0_12': {'std': 0.3517629357851265, 'mad': 0.2134964919578124},
 'nu_0_13': {'std': 0.35036903565470046, 'mad': 0.2107925679681557},
 'nu_0_14': {'std': 0.37649719535372084, 'mad': 0.22420951017790003},
 'nu_0_15': {'std': 0.3719897549797088, 'mad': 0.2172676270025704},
 'nu_0_16': {'std': 0.39047999265618194, 'mad': 0.23051234035870038},
 'nu_0_17': {'std': 0.3

In [49]:
with open("emulator_unc.json", "w") as outfile: 
    json.dump(emulator_dict, outfile)

In [51]:
with open('emulator_unc.json') as json_file:
    emulator_dict = json.load(json_file)

In [52]:
emulator_dict

{'radius': {'std': 0.0031592064924546386, 'mad': 0.0007193065020046352},
 'luminosity': {'std': 0.013091453133521488, 'mad': 0.001662317802538782},
 'star_feh': {'std': 0.0010103521414256134, 'mad': 0.0004036022869300937},
 'nu_0_6': {'std': 0.30991524050630137, 'mad': 0.18503403141781405},
 'nu_0_7': {'std': 0.3576611626309852, 'mad': 0.22224471350972408},
 'nu_0_8': {'std': 0.3701364267757371, 'mad': 0.2275139276885909},
 'nu_0_9': {'std': 0.3276912175745757, 'mad': 0.1971704275900379},
 'nu_0_10': {'std': 0.3607234429548008, 'mad': 0.22212924163957837},
 'nu_0_11': {'std': 0.3354543162205072, 'mad': 0.20247161307119654},
 'nu_0_12': {'std': 0.3517629357851265, 'mad': 0.2134964919578124},
 'nu_0_13': {'std': 0.35036903565470046, 'mad': 0.2107925679681557},
 'nu_0_14': {'std': 0.37649719535372084, 'mad': 0.22420951017790003},
 'nu_0_15': {'std': 0.3719897549797088, 'mad': 0.2172676270025704},
 'nu_0_16': {'std': 0.39047999265618194, 'mad': 0.23051234035870038},
 'nu_0_17': {'std': 0.3

## 3) json of emulator uncertainty cov matrix